In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import xarray as xr

from segysak import open_seisnc

from src.definitions import ROOT_DIR
from src.data import make_sua_surfaces

In [ ]:
%matplotlib ipympl

# Load seismic

In [ ]:
# Downloaded files directory
dst_dir = ROOT_DIR / "data"
seisnc_path = dst_dir / "interim/R3136_15UnrPrDMkD_Full_D_Rzn_RMO_Shp_vG.seisnc"

In [ ]:
seisnc = open_seisnc(seisnc_path, chunks={"inline": 100})

# Load horizons

In [ ]:
rnro1_t = make_sua_surfaces.load_mapped_horizon("rnro1_t")
ro_t = make_sua_surfaces.load_mapped_horizon("ro_t")

# Get Salt isochore

In [ ]:
salt_isochore = ro_t - rnro1_t

In [ ]:
salt_isochore

In [ ]:
inl_sel = 9100

In [ ]:
f, ax = plt.subplots(figsize=(14, 14))
pp = ax.pcolormesh(salt_isochore.cdp_x, salt_isochore.cdp_y, salt_isochore.values, cmap="viridis")
f.colorbar(pp, label="Thickness [m]")
ax.axis("equal")
ax.set_title("Salt isochore")

# Plot selected inline
selected_inline = seisnc.data.sel(iline=inl_sel)
ax.plot(
    selected_inline.cdp_x,
    selected_inline.cdp_y,
    color="blue",
    label=f"Inline: {inl_sel}"
)
ax.axis("equal")
ax.legend()

# Get reference isochrone

In [ ]:
halite_vp = 4400 # m/s
anhydrite_vp = 5900 # m/s

anhydrite_perc = 0.2
salt_vp_reference = (halite_vp*(1-anhydrite_perc) + anhydrite_vp*anhydrite_perc) 

In [ ]:
salt_vp_reference

In [ ]:
salt_isochrone_reference = salt_isochore / salt_vp_reference # seconds

# Update velocity

In [ ]:
anhydrite_perc = 0.05
salt_vp_update = (halite_vp*(1-anhydrite_perc) + anhydrite_vp*anhydrite_perc)

In [ ]:
salt_vp_update

# Update target horizon

In [ ]:
ro_t_update = rnro1_t + salt_isochrone_reference * salt_vp_update

In [ ]:
opt = dict(
    x="xline",
    y="depth",
    add_colorbar=True,
    interpolation="spline16",
    robust=True,
    yincrease=False,
    cmap="Greys",
)

f, ax = plt.subplots(figsize=(16, 10), constrained_layout=True)

seisnc.data.sel(iline=inl_sel, depth=slice(000, 6000)).plot.imshow(ax=ax, **opt)

ro_t_trace = ro_t.sel(iline=inl_sel)
ax.plot(ro_t_trace.xline, ro_t_trace.values, label="RO_T")

rnro1_t_trace = rnro1_t.sel(iline=inl_sel)
ax.plot(rnro1_t_trace.xline, rnro1_t_trace.values, label="RNRO1_T")

ro_t_update_trace = ro_t_update.sel(iline=inl_sel)
ax.plot(ro_t_update_trace.xline, ro_t_update_trace.values, label="RO_T_update")

ax.invert_xaxis()
f.legend(bbox_to_anchor=(0.8, 0.95), bbox_transform=f.transFigure)

# Build all SUA proxy surfaces

In [ ]:
make_sua_surfaces.main()

# Plot surfaces

In [ ]:
surfaces = xr.open_mfdataset(
    str(ROOT_DIR / "data/processed/surfaces/*.nc"),
    combine="nested",
    concat_dim="anhydrite_perc",
    parallel=True
)

In [ ]:
surfaces

In [ ]:
surfaces = surfaces.set_xindex(coord_names="perc")

In [ ]:
anhydrite_perc_sel = 0.05
surfaces_anhy_5perc = surfaces.sel(perc=anhydrite_perc_sel)

In [ ]:
f, ax = plt.subplots(figsize=(14, 14))
pp = ax.pcolormesh(
    surfaces_anhy_5perc.cdp_x,
    surfaces_anhy_5perc.cdp_y,
    surfaces_anhy_5perc.depth,
    cmap="viridis_r"
)
f.colorbar(pp, label="Depth [m]")
ax.axis("equal")
ax.set_title(f"RO T structure update: Anhydrite percent: {anhydrite_perc_sel}")
plt.show()

In [ ]:
opt = dict(
    x="xline",
    y="depth",
    add_colorbar=True,
    interpolation="spline16",
    robust=True,
    yincrease=False,
    cmap="Greys",
)

f, ax = plt.subplots(figsize=(16, 10), constrained_layout=True)

seisnc.data.sel(iline=inl_sel, depth=slice(0, 6000)).plot.imshow(ax=ax, **opt)

for perc in surfaces.perc.values:
    trace = surfaces.sel(perc=perc, iline=inl_sel)
    label = f"{perc:.2f}"
    ax.plot(
        trace.xline,
        trace.depth,
        label=label)

ax.invert_xaxis()
f.suptitle("Updated RO T surfaces")
f.legend(loc='lower center', ncol=9, bbox_to_anchor=(0.5, 0.05))